In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(appName='teleco-customer-churn')
spark = SparkSession.builder.getOrCreate()

This notebook will work more as a guide to help us develop the script that will run on GCP, and since we will run it localy, I'll just grab a sample of around 1000 records, so we can do it faster.

In [3]:
customers_table = spark.read.csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv', header='true', inferSchema='true')
customers_table_sample = customers_table.sample(withReplacement=False, fraction=0.15, seed=42)

In [4]:
customers_table_sample.count()

1102

We can start by applying the same steps we did on the analysis notebook to treat missing values and standardize column names.

In [5]:
customers_table_sample = customers_table_sample.withColumnRenamed('gender', 'Gender').withColumnRenamed('tenure', 'Tenure').withColumnRenamed('customerId', 'CustomerId')
customers_table_sample = customers_table_sample.replace(subset='TotalCharges', to_replace=' ', value='0.00')
customers_table_sample = customers_table_sample.withColumn('TotalCharges', customers_table_sample.TotalCharges.cast('double'))

In [6]:
customers_table_sample.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|CustomerId|Gender|SeniorCitizen|Partner|Dependents|Tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|6713-OKOMC|Female|            0|     No|        No|    10|  

In [7]:
customers_table_sample.distinct()

DataFrame[CustomerId: string, Gender: string, SeniorCitizen: int, Partner: string, Dependents: string, Tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: double, Churn: string]

### Pre-processing
---
First, we'll drop de Id column, since it doesn't present any predictive value. Then we'll convert the categorical string variables into numeric variables.

In [8]:
customers_table_sample = customers_table_sample.drop('CustomerId')

In [9]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression

In [43]:
string_variables = [variable[0] for variable in customers_table_sample.dtypes if variable[1] == 'string']
output_string_variables = [variable+'_numeric' for variable in string_variables]
rename_columns_dic = {output_string_variables[index]:string_variables[index] for index in range(len(string_variables))}

indexer_model = StringIndexer(inputCols=string_variables, outputCols=output_string_variables)
indexer_fitted = indexer_model.fit(customers_table_sample)
numeric_customers_table = indexer_fitted.transform(customers_table_sample)

numeric_customers_table = numeric_customers_table.drop(*string_variables)
numeric_customers_table = numeric_customers_table.withColumnsRenamed(rename_columns_dic)

numeric_customers_table.show()

+-------------+------+--------------+------------+------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+-----+
|SeniorCitizen|Tenure|MonthlyCharges|TotalCharges|Gender|Partner|Dependents|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|Churn|
+-------------+------+--------------+------------+------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+-----+
|            0|    10|         29.75|       301.9|   0.0|    0.0|       0.0|         1.0|          2.0|            1.0|           1.0|         0.0|             0.0|        0.0|        1.0|            0.0|     0.0|             1.0|          3.0|  0

In [24]:
numeric_customers_table.dtypes

[('SeniorCitizen', 'int'),
 ('Tenure', 'int'),
 ('MonthlyCharges', 'double'),
 ('TotalCharges', 'double'),
 ('Gender', 'double'),
 ('Partner', 'double'),
 ('Dependents', 'double'),
 ('PhoneService', 'double'),
 ('MultipleLines', 'double'),
 ('InternetService', 'double'),
 ('OnlineSecurity', 'double'),
 ('OnlineBackup', 'double'),
 ('DeviceProtection', 'double'),
 ('TechSupport', 'double'),
 ('StreamingTV', 'double'),
 ('StreamingMovies', 'double'),
 ('Contract', 'double'),
 ('PaperlessBilling', 'double'),
 ('PaymentMethod', 'double'),
 ('Churn', 'double')]

Cool, we got all the variables set as numeric values. We will now create our first model so we can use it as a baseline. I don't expect it to be the most accurate, but after that we can dig more into other pre-processing techniques.

Before we can create our model, we need to do a train-test split. I won't be using RandomSplit() as it presents some unstable results. [You can read about it here](https://sergei-ivanov.medium.com/why-you-should-not-use-randomsplit-in-pyspark-to-split-data-into-train-and-test-58576d539a36). Instead, we'll create a column with random values and filter it.

In [48]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import VectorAssembler

In [45]:
numeric_customers_table = numeric_customers_table.withColumn('train_test_index', rand(seed=42))
train_table = numeric_customers_table.filter(numeric_customers_table.train_test_index <= 0.7)
test_table = numeric_customers_table.filter(numeric_customers_table.train_test_index > 0.7)
train_table = train_table.drop('train_test_index')
test_table = test_table.drop('train_test_index')

In [46]:
print(f'train rows: {train_table.count()}\ntrain rows: {test_table.count()}')

train rows: 815
train rows: 287


In [49]:
features_cols = train_table.drop('Churn').columns
features_cols

['SeniorCitizen',
 'Tenure',
 'MonthlyCharges',
 'TotalCharges',
 'Gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [56]:
train_table.head(1)

[Row(SeniorCitizen=0, Tenure=10, MonthlyCharges=29.75, TotalCharges=301.9, Gender=0.0, Partner=0.0, Dependents=0.0, PhoneService=1.0, MultipleLines=2.0, InternetService=1.0, OnlineSecurity=1.0, OnlineBackup=0.0, DeviceProtection=0.0, TechSupport=0.0, StreamingTV=1.0, StreamingMovies=0.0, Contract=0.0, PaperlessBilling=1.0, PaymentMethod=3.0, Churn=0.0)]

In [65]:
vecAssembler = VectorAssembler(inputCols=features_cols, outputCol='features')
vectorized_table = vecAssembler.transform(train_table)
vectorized_table = vectorized_table.drop(*features_cols)
vectorized_table.show(truncate=False)

+-----+-------------------------------------------------------------------------------------+
|Churn|features                                                                             |
+-----+-------------------------------------------------------------------------------------+
|0.0  |(19,[1,2,3,7,8,9,10,14,17,18],[10.0,29.75,301.9,1.0,2.0,1.0,1.0,1.0,1.0,3.0])        |
|0.0  |[0.0,52.0,20.65,1022.95,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0] |
|0.0  |(19,[1,2,3,4,8,9,10,13,15,16,18],[52.0,79.75,4217.8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|0.0  |(19,[0,1,2,3,8,11],[1.0,43.0,90.25,3838.75,1.0,1.0])                                 |
|0.0  |[0.0,34.0,24.95,894.3,0.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0]   |
|0.0  |(19,[1,2,3,4,5,8,11,14,16],[47.0,78.9,3650.35,1.0,1.0,1.0,1.0,1.0,2.0])              |
|0.0  |[0.0,46.0,19.95,927.1,0.0,1.0,1.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,3.0]   |
|0.0  |(19,[1,2,3,4,5,6,11,12,14,17,18],[30.0,82.05,2570.2,1

In [ ]:
def create_labeled_point(row):''
    labled_point = (row['Churn'], Vectors.dense([row['']]))

In [39]:
y_train = train_table.select('Churn')
y_test = test_table.select('Churn')
x_train = train_table.drop('train_text_index')
x_test = test_table.drop('Churn', 'train_text_index')

In [66]:
lr_base_model = LogisticRegression(labelCol='Churn')
lr_trained_model = lr_base_model.fit(vectorized_table)

In [68]:
lr_trained_model.summary.accuracy

0.8098159509202454